<a href="https://colab.research.google.com/github/ror74559/Covid-19_Valenca_2020/blob/main/covid_Valen%C3%A7a_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Casos confirmados de COVID -19 em Valença-RJ
**Fonte site da prefeitura Municipal de Valença**
* Dados atualizados até o dia 10/12/2020

**Objetivo**

* Construir um gráfico de corrida de barras com a evolução dos casos confirmados em cada dia, separados por mês.

# Importando dataset

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/covid_valenca.csv')


# Preparando dados para utilizar o módulo python Bar Chart Race
*Copiei esse data set do site da Prefeitura, que tinha dados até o dia 21-11*

*Atualizei os dias que estavam faltando através dos boletins diários da prefeitura publicados no Facebook*

*Como eu preparei o csv antes de importar, não farei nenhuma limpeza de dados*

In [ ]:
df = data.filter(['Data', 'casos confirmados'])
df.head()

,Data,casos confirmados
0,23-mai.,79
1,24-mai.,79
2,25-mai.,80
3,26-mai.,81
4,27-mai.,82


Vou substituir esse formato de data para o formato YYYY-MM-DD

In [ ]:
#Dicionário para por em prática a lógica de substituição
subs = {'mai.':'2020-05','jun.':'2020-06','jul.':'2020-07','ago.':'2020-08','set.':'2020-09',
        'out.':'2020-10','nov.':'2020-11','dez.':'2020-12'}

In [ ]:
#Fiz esse teste para verificar a saída, antes de iterar com o 'for'
'''teste = '30-nov.'
traco = teste.rfind('-')
teste[traco + 1:]'''

###################################################################

"teste = '30-nov.'\ntraco = teste.rfind('-')\nteste[traco + 1:]"

In [ ]:
#Criando uma lista,chamada de data, vazia.
data = []
'''
Iteração: 
  *separador* pega o índice do '-', com isso, eu posso pegar um pedaço
da string começando do '-' ou mesmo a partir de outras posições, pegando 
o traço com referência.
  Nesse caso, eu peguei o nome do mês que começa em uma posição a mais do *separador*,
observe que o nome do mês é exatamente a chave de cada elemento do dicionário *subs*,
com isso eu vou criar os elementos da lista *data* com a junção do valor da respectiva chave
com um '-' e com o número que está antes do *serarador* em cada string.

'''
for x in df['Data']:
  separador = x.find('-')
  data.append(subs[x[separador + 1:]]+'-'+ x[:separador])
data

['2020-05-23',
 '2020-05-24',
 '2020-05-25',
 '2020-05-26',
 '2020-05-27',
 '2020-05-28',
 '2020-05-29',
 '2020-05-30',
 '2020-05-31',
 '2020-06-1',
 '2020-06-2',
 '2020-06-3',
 '2020-06-4',
 '2020-06-5',
 '2020-06-6',
 '2020-06-7',
 '2020-06-8',
 '2020-06-9',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-13',
 '2020-06-14',
 '2020-06-15',
 '2020-06-16',
 '2020-06-17',
 '2020-06-18',
 '2020-06-19',
 '2020-06-20',
 '2020-06-21',
 '2020-06-22',
 '2020-06-23',
 '2020-06-24',
 '2020-06-25',
 '2020-06-26',
 '2020-06-27',
 '2020-06-28',
 '2020-06-29',
 '2020-06-30',
 '2020-07-1',
 '2020-07-2',
 '2020-07-3',
 '2020-07-4',
 '2020-07-5',
 '2020-07-6',
 '2020-07-7',
 '2020-07-8',
 '2020-07-9',
 '2020-07-10',
 '2020-07-11',
 '2020-07-12',
 '2020-07-13',
 '2020-07-14',
 '2020-07-15',
 '2020-07-16',
 '2020-07-17',
 '2020-07-18',
 '2020-07-19',
 '2020-07-20',
 '2020-07-21',
 '2020-07-22',
 '2020-07-23',
 '2020-07-24',
 '2020-07-25',
 '2020-07-26',
 '2020-07-27',
 '2020-07-28',
 '2020-07-29'

In [ ]:
#Substituindo os elemenos da coluna 'Data' pelos elementos da lista *data*
df['Data'] = data

In [ ]:
#verificando como ficou...
df

,Data,casos confirmados
0,2020-05-23,79
1,2020-05-24,79
2,2020-05-25,80
3,2020-05-26,81
4,2020-05-27,82
...,...,...
198,2020-12-7,1275
199,2020-12-8,1320
200,2020-12-9,1350
201,2020-12-10,1375


In [ ]:
# Observando os dados do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Data               203 non-null    object
 1   casos confirmados  203 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.3+ KB


Vou mudar o tipo da coluna Data para datetime64

In [ ]:
df['Data'] = df['Data'].astype('datetime64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data               203 non-null    datetime64[ns]
 1   casos confirmados  203 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.3 KB


Modificação feita

# Casos por dia
*Esse data set possui em cada linha a soma dos casos do dia mais os casos*

 *anteriores e para contruir o gráfico de barras, que será separado por mês,*
 
*primeiro precisamos do valor de  casos por dia sem somar com os dias 
anteriores*

In [ ]:
'''A iteração a seguir, acrescenta elementos na lista *caso_dia*, iterando os elementos da coluna
'casos confirmados', da seguinte forma: pega cada elemento, a partir do segundo, o subtrai do anterior e
o resultado adiciona na lista *caso_dia*
caso_dia já possui o primeiro elemento da coluna.
'''
confirmados = df['casos confirmados']
caso_dia = [confirmados[0]]
for x in range(1, len(confirmados)):
  quant_caso = confirmados[x] - confirmados[x -1]  
  caso_dia.append(quant_caso)


In [ ]:
caso_dia

[79,
 0,
 1,
 1,
 1,
 1,
 4,
 0,
 4,
 0,
 7,
 4,
 6,
 4,
 4,
 0,
 3,
 1,
 1,
 3,
 0,
 1,
 0,
 2,
 2,
 1,
 6,
 6,
 1,
 2,
 2,
 3,
 4,
 4,
 3,
 0,
 5,
 7,
 8,
 7,
 1,
 4,
 0,
 0,
 4,
 13,
 8,
 4,
 2,
 1,
 4,
 8,
 5,
 7,
 0,
 5,
 3,
 0,
 18,
 5,
 2,
 4,
 7,
 3,
 2,
 3,
 7,
 3,
 6,
 9,
 0,
 0,
 1,
 11,
 4,
 11,
 3,
 0,
 3,
 8,
 2,
 15,
 5,
 4,
 3,
 1,
 9,
 11,
 12,
 6,
 3,
 2,
 8,
 6,
 1,
 7,
 3,
 3,
 2,
 4,
 8,
 8,
 2,
 7,
 13,
 16,
 1,
 0,
 14,
 6,
 24,
 4,
 1,
 2,
 4,
 14,
 8,
 2,
 20,
 8,
 3,
 5,
 11,
 15,
 13,
 10,
 3,
 1,
 15,
 2,
 12,
 7,
 21,
 -7,
 3,
 7,
 1,
 1,
 7,
 8,
 0,
 4,
 1,
 2,
 25,
 6,
 6,
 0,
 3,
 4,
 2,
 9,
 12,
 8,
 0,
 3,
 8,
 6,
 8,
 3,
 15,
 0,
 1,
 5,
 13,
 18,
 15,
 17,
 0,
 0,
 3,
 12,
 3,
 7,
 2,
 3,
 18,
 2,
 16,
 13,
 8,
 2,
 10,
 1,
 18,
 26,
 8,
 4,
 12,
 10,
 0,
 26,
 5,
 18,
 30,
 14,
 3,
 2,
 31,
 45,
 30,
 25,
 17]

In [ ]:
#verificando se a soma total confere com o somatório de casos
sum(caso_dia)

1392

Soma OK

Substituiundo os elemetos da coluna 'casos confirmados' pelos elementos da lista caso_dia

In [ ]:
df['casos confirmados'] = caso_dia

In [ ]:
df

,Data,casos confirmados
0,2020-05-23,79
1,2020-05-24,0
2,2020-05-25,1
3,2020-05-26,1
4,2020-05-27,1
...,...,...
198,2020-12-07,31
199,2020-12-08,45
200,2020-12-09,30
201,2020-12-10,25


# Agrupando os dados de cada mês

In [ ]:
#Criando uma lista com o nome de cada mês
meses = ['Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']

In [ ]:
#criando um dataframevazio com as colunas representando cada dia
colunas = []
for i in range(1,32):
  colunas.append(i)
df_corrida = pd.DataFrame(columns= colunas)
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31


In [ ]:
'''
  Essa iteração agrupa os dados em listas que representam cada mês e os meses que não possuem 31 dias 
  acrescentam 0 ao final. No mês de Maio são acrescentados zeros nos dias anteriores ao inicio da primeira 
  contagem registrada.
  Após cada iteração, acrescenta cada linha ao Data Frame

'''
for i in range(8):
  linha_nova = [x for x in df[df['Data'].dt.month == i + 5]['casos confirmados']]
  while len(linha_nova) < 31:
    if i == 0:
      linha_nova.insert(0,str(i))
    else:
      linha_nova.append(0)
  df_corrida.loc[len(df_corrida)] = linha_nova

In [ ]:
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79,0,1,1,1,1,4,0,4
1,0,7,4,6,4,4,0,3,1,1,3,0,1,0,2,2,1,6,6,1,2,2,3,4,4,3,0,5,7,8,0
2,7,1,4,0,0,4,13,8,4,2,1,4,8,5,7,0,5,3,0,18,5,2,4,7,3,2,3,7,3,6,9
3,0,0,1,11,4,11,3,0,3,8,2,15,5,4,3,1,9,11,12,6,3,2,8,6,1,7,3,3,2,4,8
4,8,2,7,13,16,1,0,14,6,24,4,1,2,4,14,8,2,20,8,3,5,11,15,13,10,3,1,15,2,12,0
5,7,21,-7,3,7,1,1,7,8,0,4,1,2,25,6,6,0,3,4,2,9,12,8,0,3,8,6,8,3,15,0
6,1,5,13,18,15,17,0,0,3,12,3,7,2,3,18,2,16,13,8,2,10,1,18,26,8,4,12,10,0,26,0
7,5,18,30,14,3,2,31,45,30,25,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Renomeando o índice para o nome de cada mês da lista mese criada anteriormente
for i in range(8):
  df_corrida.rename(index = ({i:meses[i]}), inplace = True)
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
Maio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79,0,1,1,1,1,4,0,4
Junho,0,7,4,6,4,4,0,3,1,1,3,0,1,0,2,2,1,6,6,1,2,2,3,4,4,3,0,5,7,8,0
Julho,7,1,4,0,0,4,13,8,4,2,1,4,8,5,7,0,5,3,0,18,5,2,4,7,3,2,3,7,3,6,9
Agosto,0,0,1,11,4,11,3,0,3,8,2,15,5,4,3,1,9,11,12,6,3,2,8,6,1,7,3,3,2,4,8
Setembro,8,2,7,13,16,1,0,14,6,24,4,1,2,4,14,8,2,20,8,3,5,11,15,13,10,3,1,15,2,12,0
Outubro,7,21,-7,3,7,1,1,7,8,0,4,1,2,25,6,6,0,3,4,2,9,12,8,0,3,8,6,8,3,15,0
Novembro,1,5,13,18,15,17,0,0,3,12,3,7,2,3,18,2,16,13,8,2,10,1,18,26,8,4,12,10,0,26,0
Dezembro,5,18,30,14,3,2,31,45,30,25,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Modelando o Data Frame para utilizar o módulo Bar Chart Race

In [ ]:
# Modificando o df_corrida para o formato transversal
df_corrida = df_corrida.T
df_corrida

,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
1,0,0,7,0,8,7,1,5
2,0,7,1,0,2,21,5,18
3,0,4,4,1,7,-7,13,30
4,0,6,0,11,13,3,18,14
5,0,4,0,4,16,7,15,3
6,0,4,4,11,1,1,17,2
7,0,0,13,3,0,1,0,31
8,0,3,8,0,14,7,0,45
9,0,1,4,3,6,8,3,30
10,0,1,2,8,24,0,12,25


In [ ]:
df_corrida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Maio      31 non-null     object
 1   Junho     31 non-null     object
 2   Julho     31 non-null     object
 3   Agosto    31 non-null     object
 4   Setembro  31 non-null     object
 5   Outubro   31 non-null     object
 6   Novembro  31 non-null     object
 7   Dezembro  31 non-null     object
dtypes: object(8)
memory usage: 3.4+ KB


Modificando o tipo de cada coluna para int64

In [ ]:
for coluna in df_corrida.columns:
  df_corrida[coluna] = df_corrida[coluna].astype('int64')
df_corrida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Maio      31 non-null     int64
 1   Junho     31 non-null     int64
 2   Julho     31 non-null     int64
 3   Agosto    31 non-null     int64
 4   Setembro  31 non-null     int64
 5   Outubro   31 non-null     int64
 6   Novembro  31 non-null     int64
 7   Dezembro  31 non-null     int64
dtypes: int64(8)
memory usage: 3.4 KB


Obtendo o acumulo diário igual ao dataset original, só que agora, em cada mês.

In [ ]:
for mes in meses:
  for dia in range(1,32):
    if dia != 1:
      df_corrida[mes][dia] += df_corrida[mes][dia - 1]

Renomeando o indice para ficar informativo no gráfico

In [ ]:
for d in range(1,32):
  df_corrida.rename(index = ({d:'Dia ' + str (d)}), inplace = True)
df_corrida


,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
Dia 1,0,0,7,0,8,7,1,5
Dia 2,0,7,8,0,10,28,6,23
Dia 3,0,11,12,1,17,21,19,53
Dia 4,0,17,12,12,30,24,37,67
Dia 5,0,21,12,16,46,31,52,70
Dia 6,0,25,16,27,47,32,69,72
Dia 7,0,25,29,30,47,33,69,103
Dia 8,0,28,37,30,61,40,69,148
Dia 9,0,29,41,33,67,48,72,178
Dia 10,0,30,43,41,91,48,84,203


# Instalando Bar Chart Race

In [ ]:
pip install bar_chart_race

     |████████████████████████████████| 163kB 6.0MB/s 


# Importando e criando o Gráfico

In [ ]:
import bar_chart_race as bcr

In [ ]:
bcr.bar_chart_race(df = pd.DataFrame(df_corrida),
                                      orientation='h',
                                      sort='desc',
                                      n_bars=8,
                                      fixed_order=False,
                                      fixed_max=True,
                                      steps_per_period=10,
                                      interpolate_period=False,
                                      label_bars=True,
                                      bar_size=.95,
                                      period_label={'x': .99, 'y': .30, 'ha': 'right', 'va': 'center'},
                                      period_length=1500, 
                                      period_summary_func=lambda v, r: {'x': .99, 'y': .02,
                                      's': f'Total de casos confirmados: {v.nlargest(8).sum():,.0f}\n\nFonte: https://www.facebook.com/PMValenca\nhttp://www.valenca.rj.gov.br               ',
                                      'ha': 'right', 'size': 7},
                                      
                                      dpi=144,
                                      cmap='dark12',
                                      title='Casos confirmados de COVID-19 por mês\nAtualizado em 11/12/2020\nValença-RJ',
                                      title_size='',
                                      bar_label_size=7,
                                      tick_label_size=7,
                                      shared_fontdict={'color' : '.1'},
                                      scale='linear',
                                      writer=None,
                                      fig=None,
                                      bar_kwargs={'alpha': .7},
                                      filter_column_colors=False)